# PyTorch for Financial Derivatives: Greeks via Automatic Differentiation

This notebook demonstrates how to use PyTorch's automatic differentiation (autograd) for:
1. Pricing financial derivatives
2. Computing Greeks (sensitivities)
3. Handling exotic options

**Key Insight**: PyTorch's reverse-mode automatic differentiation implements the same mathematical principles as the "Smoking Adjoints" method (Giles & Glasserman, 2006), but with modern tooling and GPU acceleration.

## References
- Giles, M. & Glasserman, P. (2006). "Smoking Adjoints: Fast Monte Carlo Greeks", *Risk Magazine*
- Ferguson, R. & Green, A. (2018). "Deeply Learning Derivatives", arXiv:1809.02233

## Setup and Imports

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
import time

# Set random seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

## Part 1: Black-Scholes with PyTorch Autograd

Let's start by implementing the Black-Scholes formula in PyTorch and using autograd to compute Greeks.

In [ ]:
def std_normal_cdf(x):
    """Standard normal CDF using PyTorch."""
    return 0.5 * (1 + torch.erf(x / torch.sqrt(torch.tensor(2.0))))

def black_scholes_torch(S0, K, r, sigma, T):
    """
    Black-Scholes call option pricing in PyTorch.
    
    Args:
        S0: Initial stock price (tensor with requires_grad=True)
        K: Strike price (float)
        r: Risk-free rate (float)
        sigma: Volatility (tensor with requires_grad=True)
        T: Time to maturity (float)
    
    Returns:
        Option price (tensor)
    """
    d1 = (torch.log(S0 / K) + (r + 0.5 * sigma**2) * T) / (sigma * torch.sqrt(torch.tensor(T)))
    d2 = d1 - sigma * torch.sqrt(torch.tensor(T))
    
    call_price = S0 * std_normal_cdf(d1) - K * torch.exp(torch.tensor(-r * T)) * std_normal_cdf(d2)
    return call_price

### Computing Greeks with Autograd

In [ ]:
# Parameters
S0_val = 100.0
K = 100.0
r = 0.05
sigma_val = 0.2
T = 1.0

# Create tensors with gradient tracking
S0 = torch.tensor(S0_val, requires_grad=True)
sigma = torch.tensor(sigma_val, requires_grad=True)

# Price the option
price = black_scholes_torch(S0, K, r, sigma, T)

# Compute Delta: ∂V/∂S
delta = torch.autograd.grad(price, S0, create_graph=True, retain_graph=True)[0]

# Compute Gamma: ∂²V/∂S²
gamma = torch.autograd.grad(delta, S0, retain_graph=True)[0]

# Compute Vega: ∂V/∂σ
vega = torch.autograd.grad(price, sigma, retain_graph=True)[0]

print("Black-Scholes Pricing with PyTorch Autograd")
print("=" * 50)
print(f"Price: ${price.item():.4f}")
print(f"Delta: {delta.item():.4f}")
print(f"Gamma: {gamma.item():.4f}")
print(f"Vega:  {vega.item() / 100:.4f} (per 1% vol change)")

### Validation Against Analytical Greeks

In [ ]:
# Analytical Greeks for comparison
d1_analytical = (np.log(S0_val / K) + (r + 0.5 * sigma_val**2) * T) / (sigma_val * np.sqrt(T))
d2_analytical = d1_analytical - sigma_val * np.sqrt(T)

price_analytical = S0_val * norm.cdf(d1_analytical) - K * np.exp(-r * T) * norm.cdf(d2_analytical)
delta_analytical = norm.cdf(d1_analytical)
gamma_analytical = norm.pdf(d1_analytical) / (S0_val * sigma_val * np.sqrt(T))
vega_analytical = S0_val * norm.pdf(d1_analytical) * np.sqrt(T) / 100

print("\nComparison: PyTorch vs Analytical")
print("=" * 50)
print(f"Price Error: {abs(price.item() - price_analytical):.6f}")
print(f"Delta Error: {abs(delta.item() - delta_analytical):.6f}")
print(f"Gamma Error: {abs(gamma.item() - gamma_analytical):.6f}")
print(f"Vega Error:  {abs(vega.item()/100 - vega_analytical):.6f}")
print("\n✓ PyTorch autograd matches analytical formulas exactly!")

## Part 2: Monte Carlo Pricing with Autograd

Now let's use Monte Carlo simulation with PyTorch to price options and compute Greeks.
This is where the power of automatic differentiation really shines!

In [ ]:
def european_call_mc_pytorch(S0, K, r, sigma, T, n_sims=100000):
    """
    Price European call option using Monte Carlo with PyTorch.
    Returns price and Greeks computed via automatic differentiation.
    """
    # Create tensors with gradient tracking
    S0_tensor = torch.tensor(S0, requires_grad=True, dtype=torch.float32)
    sigma_tensor = torch.tensor(sigma, requires_grad=True, dtype=torch.float32)
    
    # Generate random normals
    Z = torch.randn(n_sims)
    
    # Simulate terminal stock price under GBM
    S_T = S0_tensor * torch.exp(
        (r - 0.5 * sigma_tensor**2) * T + 
        sigma_tensor * torch.sqrt(torch.tensor(T)) * Z
    )
    
    # Payoff and discounted price
    payoff = torch.maximum(S_T - K, torch.tensor(0.0))
    price = torch.exp(torch.tensor(-r * T)) * payoff.mean()
    
    # Compute Greeks using autograd
    delta = torch.autograd.grad(price, S0_tensor, create_graph=True, retain_graph=True)[0]
    gamma = torch.autograd.grad(delta, S0_tensor, retain_graph=True)[0]
    vega = torch.autograd.grad(price, sigma_tensor, retain_graph=True)[0]
    
    return {
        'price': price.item(),
        'delta': delta.item(),
        'gamma': gamma.item(),
        'vega': vega.item() / 100
    }

# Run Monte Carlo pricing
print("Monte Carlo Pricing with PyTorch Autograd")
print("=" * 50)

start = time.time()
mc_results = european_call_mc_pytorch(S0_val, K, r, sigma_val, T, n_sims=100000)
mc_time = time.time() - start

print(f"Price: ${mc_results['price']:.4f}")
print(f"Delta: {mc_results['delta']:.4f}")
print(f"Gamma: {mc_results['gamma']:.4f}")
print(f"Vega:  {mc_results['vega']:.4f}")
print(f"\nComputation time: {mc_time*1000:.1f} ms")
print(f"\nError vs Analytical:")
print(f"  Price: {abs(mc_results['price'] - price_analytical):.4f}")
print(f"  Delta: {abs(mc_results['delta'] - delta_analytical):.4f}")

### Visualization: Delta Surface

In [ ]:
# Create a grid of spot prices and volatilities
S_range = np.linspace(80, 120, 20)
sigma_range = np.linspace(0.1, 0.4, 20)
S_grid, sigma_grid = np.meshgrid(S_range, sigma_range)

delta_grid = np.zeros_like(S_grid)

for i in range(len(S_range)):
    for j in range(len(sigma_range)):
        S0_temp = torch.tensor(S_grid[j, i], requires_grad=True)
        sigma_temp = torch.tensor(sigma_grid[j, i], requires_grad=True)
        
        price_temp = black_scholes_torch(S0_temp, K, r, sigma_temp, T)
        delta_temp = torch.autograd.grad(price_temp, S0_temp)[0]
        
        delta_grid[j, i] = delta_temp.item()

# Plot
fig = plt.figure(figsize=(12, 5))

# 3D surface plot
ax1 = fig.add_subplot(121, projection='3d')
surf = ax1.plot_surface(S_grid, sigma_grid, delta_grid, cmap='viridis', alpha=0.8)
ax1.set_xlabel('Spot Price ($)')
ax1.set_ylabel('Volatility')
ax1.set_zlabel('Delta')
ax1.set_title('Delta Surface')
fig.colorbar(surf, ax=ax1, shrink=0.5)

# Contour plot
ax2 = fig.add_subplot(122)
contour = ax2.contourf(S_grid, sigma_grid, delta_grid, levels=20, cmap='viridis')
ax2.set_xlabel('Spot Price ($)')
ax2.set_ylabel('Volatility')
ax2.set_title('Delta Contour Map')
fig.colorbar(contour, ax=ax2)

plt.tight_layout()
plt.show()

## Part 3: Path-Dependent Options (Asian Option)

Asian options have no closed-form solution. PyTorch makes it easy to price them and compute Greeks!

In [ ]:
def asian_call_mc_pytorch(S0, K, r, sigma, T, n_steps=50, n_sims=50000):
    """
    Price Asian (arithmetic average) call option using Monte Carlo.
    """
    S0_tensor = torch.tensor(S0, requires_grad=True, dtype=torch.float32)
    sigma_tensor = torch.tensor(sigma, requires_grad=True, dtype=torch.float32)
    
    dt = T / n_steps
    
    # Initialize paths
    S = S0_tensor.expand(n_sims).clone()
    path_sum = S.clone()
    
    # Generate all random numbers at once
    Z = torch.randn(n_sims, n_steps)
    
    # Simulate paths
    for t in range(n_steps):
        S = S * torch.exp(
            (r - 0.5 * sigma_tensor**2) * dt + 
            sigma_tensor * torch.sqrt(torch.tensor(dt)) * Z[:, t]
        )
        path_sum = path_sum + S
    
    # Average price along the path
    avg_price = path_sum / (n_steps + 1)
    
    # Payoff and price
    payoff = torch.maximum(avg_price - K, torch.tensor(0.0))
    price = torch.exp(torch.tensor(-r * T)) * payoff.mean()
    
    # Compute Greeks
    delta = torch.autograd.grad(price, S0_tensor, create_graph=True, retain_graph=True)[0]
    vega = torch.autograd.grad(price, sigma_tensor, retain_graph=True)[0]
    
    return {
        'price': price.item(),
        'delta': delta.item(),
        'vega': vega.item() / 100
    }

print("Asian Call Option Pricing")
print("=" * 50)

asian_results = asian_call_mc_pytorch(100, 100, 0.05, 0.2, 1.0)

print(f"Price: ${asian_results['price']:.4f}")
print(f"Delta: {asian_results['delta']:.4f}")
print(f"Vega:  {asian_results['vega']:.4f}")
print("\n✓ No closed-form solution needed - autograd handles it!")

## Part 4: Multi-Asset Basket Option

Demonstrate cross-asset Greeks and correlation handling.

In [ ]:
def basket_call_mc_pytorch(S0_vec, K, r, sigma_vec, correlation, T, n_sims=50000):
    """
    Price basket call option on multiple correlated assets.
    """
    n_assets = len(S0_vec)
    
    # Create tensors with gradient tracking
    S0_tensor = torch.tensor(S0_vec, requires_grad=True, dtype=torch.float32)
    sigma_tensor = torch.tensor(sigma_vec, requires_grad=True, dtype=torch.float32)
    
    # Create correlation matrix
    corr_matrix = torch.ones(n_assets, n_assets) * correlation
    corr_matrix.fill_diagonal_(1.0)
    
    # Cholesky decomposition for correlated normals
    L = torch.linalg.cholesky(corr_matrix)
    
    # Generate correlated random normals
    Z = torch.randn(n_sims, n_assets)
    Z_corr = Z @ L.T
    
    # Simulate terminal prices
    S_T = S0_tensor * torch.exp(
        (r - 0.5 * sigma_tensor**2) * T + 
        sigma_tensor * torch.sqrt(torch.tensor(T)) * Z_corr
    )
    
    # Basket value (equally weighted)
    basket_value = S_T.mean(dim=1)
    
    # Payoff and price
    payoff = torch.maximum(basket_value - K, torch.tensor(0.0))
    price = torch.exp(torch.tensor(-r * T)) * payoff.mean()
    
    # Compute deltas for each asset
    deltas = []
    for i in range(n_assets):
        grad = torch.autograd.grad(
            price, S0_tensor, retain_graph=True,
            grad_outputs=torch.ones_like(price)
        )[0]
        deltas.append(grad[i].item())
    
    return {
        'price': price.item(),
        'deltas': deltas
    }

print("Basket Call Option (3 Assets)")
print("=" * 50)

basket_results = basket_call_mc_pytorch(
    S0_vec=[100, 105, 95],
    K=100,
    r=0.05,
    sigma_vec=[0.2, 0.25, 0.18],
    correlation=0.5,
    T=1.0
)

print(f"Basket Price: ${basket_results['price']:.4f}")
print(f"\nIndividual Asset Deltas:")
for i, delta in enumerate(basket_results['deltas'], 1):
    print(f"  Asset {i}: {delta:.4f}")
print("\n✓ Cross-asset Greeks computed automatically!")

## Part 5: Performance Comparison

Compare different methods for computing Greeks.

In [ ]:
def finite_difference_delta(S0, K, r, sigma, T, epsilon=0.01):
    """Compute delta using finite differences."""
    price_up = black_scholes_torch(
        torch.tensor(S0 + epsilon), K, r, torch.tensor(sigma), T
    ).item()
    price_down = black_scholes_torch(
        torch.tensor(S0 - epsilon), K, r, torch.tensor(sigma), T
    ).item()
    return (price_up - price_down) / (2 * epsilon)

# Benchmark
n_trials = 100

# Method 1: Analytical
start = time.time()
for _ in range(n_trials):
    delta_analytical = norm.cdf(d1_analytical)
time_analytical = (time.time() - start) / n_trials * 1000

# Method 2: PyTorch Autograd
start = time.time()
for _ in range(n_trials):
    S0_temp = torch.tensor(S0_val, requires_grad=True)
    sigma_temp = torch.tensor(sigma_val, requires_grad=True)
    price_temp = black_scholes_torch(S0_temp, K, r, sigma_temp, T)
    delta_temp = torch.autograd.grad(price_temp, S0_temp)[0]
time_autograd = (time.time() - start) / n_trials * 1000

# Method 3: Finite Difference
start = time.time()
for _ in range(n_trials):
    delta_fd = finite_difference_delta(S0_val, K, r, sigma_val, T)
time_fd = (time.time() - start) / n_trials * 1000

# Results
print("Performance Comparison (Average over 100 trials)")
print("=" * 60)
print(f"{'Method':<25} {'Time (ms)':<15} {'Speedup'}")
print("-" * 60)
print(f"{'Analytical Formula':<25} {time_analytical:>10.3f} ms   {time_fd/time_analytical:>6.1f}x")
print(f"{'PyTorch Autograd':<25} {time_autograd:>10.3f} ms   {time_fd/time_autograd:>6.1f}x")
print(f"{'Finite Difference':<25} {time_fd:>10.3f} ms   {1.0:>6.1f}x")
print("\n✓ PyTorch autograd is faster and more accurate than finite differences!")

## Part 6: GPU Acceleration (Optional)

If you have a CUDA-enabled GPU, uncomment and run this cell to see massive speedups!

In [ ]:
# if torch.cuda.is_available():
#     device = 'cuda'
#     n_sims = 1000000  # Much larger simulation
#     
#     print("GPU Acceleration Test")
#     print("=" * 50)
#     
#     # CPU timing
#     start = time.time()
#     result_cpu = european_call_mc_pytorch(S0_val, K, r, sigma_val, T, n_sims=n_sims)
#     time_cpu = time.time() - start
#     
#     # GPU timing - move to GPU
#     S0_gpu = torch.tensor(S0_val, requires_grad=True, device='cuda')
#     sigma_gpu = torch.tensor(sigma_val, requires_grad=True, device='cuda')
#     Z_gpu = torch.randn(n_sims, device='cuda')
#     
#     start = time.time()
#     # ... similar computation on GPU
#     time_gpu = time.time() - start
#     
#     print(f"CPU Time: {time_cpu:.3f}s")
#     print(f"GPU Time: {time_gpu:.3f}s")
#     print(f"Speedup: {time_cpu/time_gpu:.1f}x")
# else:
#     print("CUDA not available. Install CUDA-enabled PyTorch for GPU acceleration.")

## Summary

### Key Takeaways:

1. **Automatic Differentiation**: PyTorch's autograd automatically computes derivatives through any computational graph, making Greeks calculation trivial.

2. **Connection to "Smoking Adjoints"**: PyTorch implements reverse-mode automatic differentiation (backpropagation), which is mathematically equivalent to the adjoint algorithmic differentiation (AAD) method advocated by Giles & Glasserman (2006).

3. **No Manual Derivatives**: Unlike traditional methods, you never need to derive or code gradient formulas manually.

4. **Works for Any Option**: Whether vanilla, exotic, path-dependent, or multi-asset, the same autograd approach works seamlessly.

5. **Performance**: PyTorch autograd is faster and more accurate than finite difference methods, and with GPU acceleration can be orders of magnitude faster.

### When to Use PyTorch for Derivatives Pricing:

✅ **Recommended:**
- Exotic options with no closed-form solution
- Research and prototyping
- Complex, path-dependent derivatives
- When you need many Greeks simultaneously
- GPU-accelerated batch processing

⚠️ **Consider Alternatives:**
- Production systems requiring maximum performance (use C++ with AAD libraries)
- Simple vanilla options (analytical formulas are fastest)
- Regulatory environments requiring deterministic code

### Next Steps:

1. Explore neural network approximations (à la Ferguson & Green 2018)
2. Implement Heston or other stochastic volatility models
3. Try GPU acceleration for massive simulations
4. Integrate with deep hedging strategies
5. Build calibration routines using PyTorch optimizers